<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_%EC%A2%85%ED%95%A9_%EB%93%80%EC%96%BC%EB%AA%A8%EB%A9%98%ED%85%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [2]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.4/191.4 kB 7.3 MB/s eta 0:00:00


In [3]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
start_date = today - timedelta(days=375)

# 전략명
str_strategy = '종합 듀얼모멘텀 '

# 전략에 사용할 ETF 명 리스트에 담기
etf_shy = ['SHY']
etf_stock = ['SPY', 'EFA']
etf_bond = ['LQD', 'HYG']
etf_reits = ['VNQ', 'REM']
etf_stress = ['TLT', 'GLD']

# 자산군 리스트 더하기, 중복 제거
etf_order = etf_shy + etf_stock + etf_bond + etf_reits + etf_stress

# ETF 가격 추출
df_etf = yf.download(etf_order, start=start_date, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0)

display(df_etf)

[*********************100%%**********************]  9 of 9 completed


,SHY,SPY,EFA,LQD,HYG,VNQ,REM,TLT,GLD
2023-09-01,80.9900,451.1900,71.4900,105.0300,75.0700,82.5200,24.3000,94.8500,180.1100
2023-08-31,81.0340,450.3500,71.5300,105.7830,74.9290,82.3800,24.1100,96.3510,180.0200
2023-08-30,80.9542,451.0100,71.8100,105.6036,74.9788,82.9900,24.0600,95.9223,180.3200
2023-08-29,80.9542,449.1600,71.8500,105.7531,75.0086,82.6900,24.1600,96.0220,179.8700
2023-08-28,80.7747,442.7600,70.9700,104.9161,74.5408,81.7000,23.8300,95.0350,178.1300
...,...,...,...,...,...,...,...,...,...
2022-08-30,80.1047,391.8118,60.7582,105.8963,70.4004,89.2477,25.6733,109.3000,160.5100
2022-08-29,80.1436,396.1608,61.1983,105.7621,70.9453,90.4988,26.2873,109.0291,161.8600
2022-08-26,80.2020,398.7977,61.3352,106.5484,71.2177,91.3201,26.4228,109.9386,161.7600
2022-08-25,80.2215,412.7696,62.9882,107.1909,72.4108,93.6791,26.8111,109.1161,163.7500


In [5]:
# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} {str_strategy}.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')

# 월평균영업일수. average Monthly Business Day
MBD = 21

# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)

### 2023-09-03 종합 듀얼모멘텀 ### ETF 현재-1-3-6-12 개월 가격

,SHY,SPY,EFA,LQD,HYG,VNQ,REM,TLT,GLD
today/2023-09-01,80.9900,451.1900,71.4900,105.0300,75.0700,82.5200,24.3000,94.8500,180.1100
1mo_ago/2023-08-03,80.7048,448.8400,71.9600,104.4378,73.8640,83.7100,23.6300,94.5664,179.5400
3mo_ago/2023-06-02,80.7558,426.3363,71.3422,106.2201,73.4837,81.0448,21.9295,101.1317,181.0500
6mo_ago/2023-03-03,79.5867,401.1631,69.6041,104.0726,72.6237,85.1063,23.1280,100.2568,172.4900
12mo_ago/2022-08-31,80.0949,388.8305,60.3474,104.7840,70.0152,88.8275,25.6642,108.2550,159.2700


In [6]:
## DataFrame 만들기: 수익률
ror_1mo = df_etf.pct_change(-MBD).iloc[0:1]*100    # 1개월 수익률
ror_3mo = df_etf.pct_change(-MBD*3).iloc[0:1]*100    # 3개월 수익률
ror_6mo = df_etf.pct_change(-MBD*6).iloc[0:1]*100   # 6개월 수익률
ror_12mo = df_etf.pct_change(-MBD*12).iloc[0:1]*100  # 12개월 수익률

df_etf_ror = pd.concat([ror_1mo, ror_3mo, ror_6mo, ror_12mo])
df_etf_ror.index = df_momentum_index[1:]

str_etf_ror = str_today + str_strategy + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)

### 2023-09-03 종합 듀얼모멘텀 ### ETF 1-3-6-12 개월 수익률

,SHY,SPY,EFA,LQD,HYG,VNQ,REM,TLT,GLD
1mo_ago/2023-08-03,0.3533,0.5236,-0.6531,0.5670,1.6327,-1.4216,2.8354,0.2999,0.3175
3mo_ago/2023-06-02,0.2900,5.8296,0.2071,-1.1204,2.1588,1.8202,10.8099,-6.2114,-0.5192
6mo_ago/2023-03-03,1.7632,12.4705,2.7095,0.9199,3.3684,-3.0389,5.0674,-5.3930,4.4176
12mo_ago/2022-08-31,1.1175,16.0377,18.4641,0.2348,7.2195,-7.1008,-5.3157,-12.3828,13.0847


In [13]:
display(Markdown(str_today[1:] + str_strategy))

# 주식: SPY, EFA 둘 다 SHY 보다 수익률 낮은지 확인
str_is_stock_lower_then_shy = 'False'

# 둘다 다 낮으면 bond로
if (df_etf_ror['SPY'].iloc[3] < df_etf_ror['SHY'].iloc[3]) and (df_etf_ror['EFA'].iloc[3] < df_etf_ror['SHY'].iloc[3]):
  str_is_stock_lower_then_shy = 'True'

str_etf = "## 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 미국 단기채(SHY) 보다 낮은가? " + str_is_stock_lower_then_shy
display(Markdown(str_etf))

# 아니라면 SPY, EFA 중 12개월 수익률이 더 높은 것 표시
str_higher_stock = ''

if str_is_stock_lower_then_shy == 'True':
  str_higher_stock = 'SHY'
  str_higher_stock_text = "## 투자할 주식 대체품: " + str_higher_stock
else:
  if df_etf_ror['SPY'].iloc[3] > df_etf_ror['EFA'].iloc[3]:
    str_higher_stock = 'SPY'
  else:
    str_higher_stock = 'EFA'
  str_higher_stock_text = "## 투자할 주식: " + str_higher_stock
display(Markdown(str_higher_stock_text))

# 채권: LQD, HYG 중 12개월 수익률 더 높은 것 표시
str_higher_bond = ''
if df_etf_ror['LQD'].iloc[3] > df_etf_ror['HYG'].iloc[3]:
  str_higher_bond = 'LQD'
else:
  str_higher_bond = 'HYG'

str_higher_bond_text = "## 투자할 채권: " + str_higher_bond
display(Markdown(str_higher_bond_text))

# 부동산: VNQ, REM 중 12개월 수익률 더 높은 것 표시
str_higher_reits = ''
if df_etf_ror['VNQ'].iloc[3] > df_etf_ror['REM'].iloc[3]:
  str_higher_reits = 'VNQ'
else:
  str_higher_reits = 'REM'

str_higher_reits_text = "## 투자할 부동산: " + str_higher_reits
display(Markdown(str_higher_reits_text))

# 스트레스: TLT, GLD 중 12개월 수익률 더 높은 것 표시
str_higher_stress = ''
if df_etf_ror['TLT'].iloc[3] > df_etf_ror['GLD'].iloc[3]:
  str_higher_stress = 'TLT'
else:
  str_higher_stress = 'GLD'

str_higher_stress_text = "## 투자할 스트레스: " + str_higher_stress
display(Markdown(str_higher_stress_text))


# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_etf}\n')
  f.write(f'{str_higher_stock_text}\n')
  f.write(f'{str_higher_bond_text}\n')
  f.write(f'{str_higher_reits_text}\n')
  f.write(f'{str_higher_stress_text}\n')

## 2023-09-03 종합 듀얼모멘텀 

## 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 미국 단기채(SHY) 보다 낮은가? False

## 투자할 주식: EFA

## 투자할 채권: HYG

## 투자할 부동산: REM

## 투자할 스트레스: GLD